#### Installing packages

In [1]:
!pip install streamlit -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.3/79.3 kB 3.5 MB/s eta 0:00:00


In [2]:
!pip install pyngrok

In [3]:
pip install scikit-learn==1.2.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 53.1 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.5.2
    Uninstalling scikit-learn-1.5.2:
      Successfully uninstalled scikit-learn-1.5.2


In [4]:
pip install joblib==1.2.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 4.7 MB/s eta 0:00:00
  Attempting uninstall: joblib
    Found existing installation: joblib 1.4.2
    Uninstalling joblib-1.4.2:
      Successfully uninstalled joblib-1.4.2


In [5]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [6]:
# import packages
import pandas as pd
import joblib
import streamlit as st
import numpy as np
from pyngrok import ngrok
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import chi2, SelectKBest
from sklearn.preprocessing import LabelEncoder

In [7]:
 def disease_selection():
  df = pd.read_csv("/content/drive/MyDrive/High_LP_Scan_AP_Comorbidities_Dataset.csv")

  disease_dummies = pd.get_dummies(df['long_title'])
  data = pd.concat([df, disease_dummies], axis=1).drop('long_title', axis=1)

  # aggregate all the rows into unique patients based on subject_id
  aggregated_data = data.groupby('subject_id').agg({
      'lipase_level': 'first',
      'age': 'first',
      'gender': 'first',
      'race': 'first',
      'in_hospital_death': 'max',  # If death recorded in any row, we consider the patient deceased
      'length_of_stay': 'first',
      **{col: 'max' for col in disease_dummies.columns}  # max for binary disease presence across multiple rows
  }).reset_index()

  # grouping all variants of AP into single column
  columns_to_drop = [col for col in aggregated_data.columns if 'acute pancreatitis' in col.lower()]

  aggregated_data.drop(columns = columns_to_drop, inplace = True)

  X = aggregated_data.drop(['subject_id', 'in_hospital_death'], axis = 1)

  y = aggregated_data['in_hospital_death'].astype(int)  # Converting boolean to int

  X_encoded = X.apply(LabelEncoder().fit_transform)

# Apply Chi-squared test
  chi2_features = SelectKBest(score_func = chi2, k = 'all')  # 'all' to test each feature
  chi2_features.fit(X_encoded, y)
  chi2_scores = pd.DataFrame(chi2_features.scores_, columns = ['score'], index = X.columns)
  chi2_p_values = pd.DataFrame(chi2_features.pvalues_, columns = ['p-value'], index = X.columns)
  chi2_results = pd.concat([chi2_scores, chi2_p_values], axis=1)

  significant_features = chi2_results[chi2_results['p-value'] < 0.05]
  return list(significant_features.index)[3:]+["Acute pancreatitis"]

In [8]:
model = joblib.load('/content/drive/MyDrive/SVMC_small.joblib')
data = {
        'age': [80],
        'gender': ['M'],
        'race': ["WHITE"],
        'lipase_level': [2333.4],
        'length_of_stay': [65],
    }

disease_list = disease_selection()

selected_diseases = ['Acute pancreatitis']
    # Add diseases as binary features
for disease in disease_list:
  data[disease] = [True if disease in selected_diseases else False]

features = pd.DataFrame(data)

model.predict_proba(features)[:, 1][0]

0.00015693968605793614

In [9]:
model

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['lipase_level', 'age',
                                                   'length_of_stay']),
                                                 ('binary', 'passthrough',
                                                  ['Abdominal aneurysm without '
                                                   'mention of rupture',
                                                   'Abnormal uterine and '
                                                   'vaginal bleeding, '
                                                   'unspecified',
                                                   'Acute (reversible) '
                                                   'ischemia of large '
                                                   'intestine, extent '
                                                   'unspecified',
                                                   'Acute and subacute '
                                                   'infective end...
                                                   'specified sites',
                                                   'Cardiac catheterization as '
                                                   'the cause of abnormal '
                                                   'reaction of patient, or of '
                                                   'later complication, '
                                                   'without mention of '
                                                   'misadventure at time of '
                                                   'procedure',
                                                   'Cardiac pacemaker in situ',
                                                   'Chronic atrial '
                                                   'fibrillation', ...]),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['gender', 'race'])])),
                ('smote', SMOTE(random_state=42)),
                ('classifier', SVC(probability=True))])

In [10]:
%%writefile app.py

import pandas as pd
import joblib
import streamlit as st
import numpy as np
from pyngrok import ngrok
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import chi2, SelectKBest
from sklearn.preprocessing import LabelEncoder

# Load your trained model
model = joblib.load('/content/drive/MyDrive/SVMC_small.joblib')

def add_bg_from_gradient():
    st.markdown(
        """
        <style>
        .stApp {
            background: linear-gradient(180deg, rgba(58,123,213,1) 0%, rgba(0,210,255,1) 100%);
            color: #ffffff;
        }
        </style>
        """,
        unsafe_allow_html=True
    )

add_bg_from_gradient()

image_path = '/content/drive/MyDrive/DALL·E 2024-10-28 23.05.16 - A haunting animated scene depicting acute pancreatitis and its effects, with human anatomy as a central focus, surrealistic elements to show disease p.webp'  # Local path

    # Display the image
st.image(image_path, caption='Death due to Acute Pancreatitis & other diseases', width = 300)

def disease_selection():
  df = pd.read_csv("/content/drive/MyDrive/High_LP_Scan_AP_Comorbidities_Dataset.csv")

  disease_dummies = pd.get_dummies(df['long_title'])
  data = pd.concat([df, disease_dummies], axis=1).drop('long_title', axis=1)

  # aggregate all the rows into unique patients based on subject_id
  aggregated_data = data.groupby('subject_id').agg({
      'lipase_level': 'first',
      'age': 'first',
      'gender': 'first',
      'race': 'first',
      'in_hospital_death': 'max',  # If death recorded in any row, we consider the patient deceased
      'length_of_stay': 'first',
      **{col: 'max' for col in disease_dummies.columns}  # max for binary disease presence across multiple rows
  }).reset_index()

  # grouping all variants of AP into single column
  columns_to_drop = [col for col in aggregated_data.columns if 'acute pancreatitis' in col.lower()]

  aggregated_data.drop(columns = columns_to_drop, inplace = True)

  X = aggregated_data.drop(['subject_id', 'in_hospital_death'], axis = 1)

  y = aggregated_data['in_hospital_death'].astype(int)  # Converting boolean to int

  X_encoded = X.apply(LabelEncoder().fit_transform)

# Apply Chi-squared test
  chi2_features = SelectKBest(score_func = chi2, k = 'all')  # 'all' to test each feature
  chi2_features.fit(X_encoded, y)
  chi2_scores = pd.DataFrame(chi2_features.scores_, columns = ['score'], index = X.columns)
  chi2_p_values = pd.DataFrame(chi2_features.pvalues_, columns = ['p-value'], index = X.columns)
  chi2_results = pd.concat([chi2_scores, chi2_p_values], axis=1)

  significant_features = chi2_results[chi2_results['p-value'] < 0.05]
  return list(significant_features.index)[3:]+["Acute pancreatitis"]

disease_list = disease_selection()

# Function to make predictions
def predict_mortality(input_data):
    probability_of_death = model.predict_proba(input_data)[:, 1]
    return probability_of_death[0]

# Creating the sidebar for input
st.sidebar.header('Patient Input Features')
def user_input_features():
    age = st.sidebar.slider('Age', 0, 120, 50)
    gender = st.sidebar.selectbox('Gender', ('M', 'F'))
    race = st.sidebar.selectbox('Race', set(pd.read_csv('/content/drive/MyDrive/High_LP_Scan_AP_Comorbidities_Dataset.csv')['race']))
    lipase_level = st.sidebar.number_input('Lipase Level', min_value=100.0, max_value=30000.0, value=1000.0)
    length_of_stay = st.sidebar.number_input('Length of Stay', min_value=0.0, max_value=365.0, value=5.0)

    # Dynamic disease selection using multiselect
    selected_diseases = st.sidebar.multiselect('Select Diseases', disease_list)

    # Create DataFrame from input data
    data = {
        'age': [age],
        'gender': [gender],
        'race': [race],
        'lipase_level': [lipase_level],
        'length_of_stay': [length_of_stay],
    }

    # Add diseases as binary features
    for disease in disease_list:
        data[disease] = [True if disease in selected_diseases else False]

    features = pd.DataFrame(data)
    return features

input_df = user_input_features()

# Display the input features
st.header('Specified Input features')
st.write(input_df)

# Prediction button
if st.button('Predict Mortality Risk'):
    prediction = predict_mortality(input_df)
    st.header('Prediction')
    st.write(f'Probability of Death: {prediction:.2%}')


Writing app.py


In [11]:
ngrok.set_auth_token("")

In [13]:
# Start Streamlit app
import os
os.system("streamlit run app.py &")

# Open a ngrok tunnel to the streamlit port
public_url = ngrok.connect(8501)
print(f"Streamlit app is live at {public_url}")

Streamlit app is live at NgrokTunnel: "https://c155-35-227-74-54.ngrok-free.app" -> "http://localhost:8501"


In [12]:
tunnels = ngrok.get_tunnels()
tunnels

# try:
#     # Block until CTRL-C or some other terminating event
#     ngrok_process.proc.wait()
# except KeyboardInterrupt:
#     print(" Shutting down server.")

ngrok.kill()